Build a regression model.

In [21]:
import sqlite3
import pandas as pd
import statsmodels.api as sm

In [23]:
# Define the database path
db_path = r"C:\LHL\LHL-Statistical-Modelling\LHL-Statistical-Modelling\Database\data.db"

In [26]:
# Connect to SQLite database
conn = sqlite3.connect(db_path)

try:
    # Load only relevant columns from bars and bike_stations tables
    query = """
    SELECT 
        bars.Yelp_Rating, 
        bars.Reviews, 
        bike_stations.Total_Slots
    FROM bars
    JOIN bike_stations ON bars.Bike_Station = bike_stations.Name
    """

    # Load data into Pandas DataFrame
    df = pd.read_sql(query, conn)

finally:
    # Close the database after data is loaded
    conn.close()

# Print dataset shape before processing
print("Data shape before filtering:", df.shape)

# Removes rows where Yelp_Rating or Reviews are NULL or 0
df = df.dropna(subset=["Yelp_Rating", "Reviews"])
df = df[(df["Yelp_Rating"] > 0) & (df["Reviews"] > 0)]

# Print dataset shape after filtering
print("Data shape after filtering:", df.shape)

# Define independent (X) and dependent (Y) variables
X = df[["Reviews", "Yelp_Rating"]]
Y = df["Total_Slots"]

# Add a constant to the model (for the intercept term)
X = sm.add_constant(X)

# Ensure data isn't empty before running regression
if X.shape[0] == 0 or Y.shape[0] == 0:
    print("🚨 ERROR: Dataset is empty after filtering. Check SQL query or data filtering.")
else:
    # Fit the OLS regression model
    model = sm.OLS(Y, X).fit()
    
    # Display regression results summary
    print(model.summary())

Data shape before filtering: (16893, 3)
Data shape after filtering: (1883, 3)
                            OLS Regression Results                            
Dep. Variable:            Total_Slots   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     6.567
Date:                Mon, 10 Feb 2025   Prob (F-statistic):            0.00144
Time:                        17:35:51   Log-Likelihood:                -5995.9
No. Observations:                1883   AIC:                         1.200e+04
Df Residuals:                    1880   BIC:                         1.201e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

# **Regression Results Interpretation**

## **1. R-squared (0.007) – Model Fit**
- The **R-squared value is very low (0.007)**, meaning the model explains only **0.7% of the variance** in `Total_Slots` based on `Reviews` and `Yelp_Rating`.  
- This suggests that **these two variables are poor predictors** for determining the total number of slots at a bike station.  
`Total_Slots`.

---

## **2. P-values – Statistical Significance of Variables**
| Variable       | Coefficient | P-value  | Interpretation |
|---------------|------------|---------|---------------|
| **Reviews**   | -0.0477    | **0.010** | Statistically significant (p < 0.05). More reviews are slightly associated with **fewer total slots**, but the effect is small. |
| **Yelp_Rating** | -0.4908 | **0.012** | Statistically significant (p < 0.05). Higher Yelp ratings are associated with **fewer total slots** at nearby bike stations. |
| **Intercept** | 9.2190    | **0.000** | Significant, meaning the model’s baseline estimate (when `Reviews` and `Yelp_Rating` are zero) is reliable. |

---

## **3. Model Fit & F-Statistic**
- **F-statistic: 6.567, p = 0.00144** → The model **as a whole** is statistically significant.
- **However, given the very low R-squared**, even though the model is statistically valid, it has **low predictive power**.

---

## **4. Implications & Next Steps**
- **The model is weak** → `Reviews` and `Yelp_Rating` do not strongly explain `Total_Slots`.

---

# Stretch

How can you turn the regression model into a classification model?

## **1. Problem Statement**
The current regression model predicts a **continuous variable** (`Total_Slots`). To convert this into a **classification model**, need to **group `Total_Slots` into categories** and use a classification algorithm.

---

## **2. Steps to Convert Regression to Classification**
### **Step 1: Define Classes for `Total_Slots`**
Since `Total_Slots` is continuous, we will categorize it into **three discrete classes**:
- **Low Capacity (`0`)** → `Total_Slots < 10`
- **Medium Capacity (`1`)** → `10 ≤ Total_Slots ≤ 15`
- **High Capacity (`2`)** → `Total_Slots > 15`

This transforms the **regression target (`Total_Slots`) into a classification label**.

---

### **Step 2: Update the Target Variable (`Y`)**
- Instead of predicting an exact slot number, we now **predict a category** (`Low`, `Medium`, `High`).
- We apply a function that **assigns each row to one of the three classes** based on its `Total_Slots` value.

---

### **Step 3: Choose a Classification Model**
**Logistic Regression**, which works well for **multi-class classification**.

- **Why Logistic Regression?**  
  - Simple and interpretable.
  - Works well when the classes are **linearly separable**.
  - Requires minimal hyperparameter tuning.

---